### Summary

Refactor the Linear_solvers package, so that the quantum algorithm (HHL) accepts oracles for the different blocks and computes de different parameters according to the error bounds calculated in [1].

### Workflow

Since from the quantum algorithm one can only compute functions from the solution vector, an instance of the algorithm can be created by specifying the error tolerance and the desired observable(s) to be computed:
```python
from qiskit.aqua.algorithms.linear_solvers.hhl import HHL
from qiskit.aqua.algorithms.linear_solvers.observables import EuclideanNorm, AbsoluteAverage

# initialise the algorithm
hhl = HHL(observable=[EuclideanNorm(), MatrixFunctional()], epsilon=1e-3)
```

However some observables will depend on the dimension of the system, which ideally one only specifies one `hhl.solve` is called. Therefore, we could allow to add observables from this method as well.
```python
from qiskit.aqua.algorithms.linear_solvers.observables import MatrixFunctional

# specify the problem
matrix = [[1/2, -2], [-2, 1/2]]
vector = [1, 0]
# run the algorithm and add an observable which depends on the dimension 
result = hhl.solve(matrix=matrix, vector=vector, observable=MatrixFunctional([[1,2],[2,1]]))

# array with the calculated values for each observable
observable_values = result.value

```

### Questions

When to specify observables: within HHL `__init__`, `solve`, or both?

### Future work

Add the Richardson extrapolation logic.

### References
[1] Carrera Vazquez, A., Hiptmair, R., & Woerner, S. (2020). Enhancing the Quantum Linear Systems Algorithm using Richardson Extrapolation. `arXiv:2009.04484 <http://arxiv.org/abs/2009.04484>`